In [6]:
import os 
import json

# Load keys from a config file
config_file_path = os.path.join(".", "config_llm.json")
with open(config_file_path, "r") as config_file:
    config = json.load(config_file)

os.environ['LANGCHAIN_TRACING_V2'] = config.get("LANGCHAIN_TRACING_V2", "false")
os.environ['LANGCHAIN_ENDPOINT'] = config.get("LANGCHAIN_ENDPOINT", "https://api.langchain.com")
os.environ["GOOGLE_API_KEY"] = config.get("GOOGLE_API_KEY", "")
os.environ['LANGCHAIN_API_KEY'] = config.get("LANGCHAIN_API_KEY", "")

In [8]:
import requests
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

c:\Users\Pankaj rawat\IdeaProjects\getInterviews\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
